In [1]:
pip install torch torch-scatter torch-spline-conv torch-cluster torch_sparse torch_geometric

  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "c:\Users\Sheri\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\Sheri\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Sheri\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Sheri\AppData\Local\Temp\pip-build-env-d501xqph\

In [76]:
import numpy as np
import pandas as pd
import os
import sklearn as sk
import torch
from torch_sparse import SparseTensor
from scipy import sparse

ModuleNotFoundError: No module named 'torch_sparse'

In [14]:
import pandas as pd

def get_friends(userID : str, df : pd.DataFrame):
    """Returns a list containing the friends of userID in df. df needs columns 'userID', 'friendID' (usually user_friends)."""
    friends = df.loc[df['userID'] == userID]
    return [x for x in friends['friendID']]

def get_tags(userID : str, df: pd.DataFrame):
    """Returns a list containing the tags that userID has assigned to artists in df. df needs columns 'userID', 'tagID' (usually user_taggedartists)."""
    tags = df.loc[df['userID'] == userID]
    return [x for x in tags['tagID']]

def get_artists(userID : str, df : pd.DataFrame):
    """Returns a list containing the artists that userID has listened to in df. df needs columns 'userID', 'artistID' (usually user_artists)."""
    artists = df.loc[df['userID'] == userID]
    return [x for x in artists['artistID']]

def get_df_name(df : pd.DataFrame, globals : dict): # From https://stackoverflow.com/questions/31727333/get-the-name-of-a-pandas-dataframe
    """Returns the name of the input dataframe as a string. Solely used for nice formatting. Requires input of globals() dict."""
    name =[x for x in globals if globals[x] is df][0]
    return name

In [15]:
cwd = os.getcwd()
artists = pd.read_csv(os.path.join(cwd,'data','artists.dat'), delimiter='\t')
tags = pd.read_csv(os.path.join(cwd,'data','tags.dat'), delimiter='\t',encoding='ISO-8859-1')
user_artists = pd.read_csv(os.path.join(cwd,'data','user_artists.dat'), delimiter='\t')
user_friends = pd.read_csv(os.path.join(cwd,'data','user_friends.dat'), delimiter='\t')
user_taggedartists_timestamps = pd.read_csv(os.path.join(cwd,'data','user_taggedartists-timestamps.dat'), delimiter='\t')
user_taggedartists = pd.read_csv(os.path.join(cwd,'data','user_taggedartists.dat'), delimiter='\t')

In [16]:
users = user_artists['userID'].unique()
singleartistusers = [user for user in users if len(get_artists(user,user_artists)) == 1]
singleartistusersdf = user_artists[user_artists['userID'].isin(singleartistusers)]
user_artists_temp = user_artists[~user_artists['userID'].isin(singleartistusers)]

In [17]:
from sklearn.model_selection import train_test_split

user_artists_train, user_artists_test = train_test_split(user_artists_temp, test_size = 0.2, stratify = user_artists_temp['userID'], random_state = 47)

user_artists_train = pd.concat([user_artists_train,singleartistusersdf])

In [18]:
user_taggedartists_test = user_taggedartists.merge(user_artists_test[['userID','artistID']], on = ['userID','artistID'], how = 'inner')
user_taggedartists_train = user_taggedartists.merge(user_artists_test[['userID','artistID']], on = ['userID','artistID'], how = 'left', indicator = True)
user_taggedartists_train = user_taggedartists_train[user_taggedartists_train['_merge'] == 'left_only'].drop(columns = ['_merge'])

In [19]:
filepath = os.path.join(cwd,'SheridanH','data')
for df in [user_artists_train,user_artists_test,user_taggedartists_train,user_taggedartists_test]:
    df.to_csv(os.path.join(filepath,get_df_name(df, globals()) + '.csv'))

## Adjacency Matrix

### Interaction Matrix

In [42]:
user_vertices = user_artists['userID'].unique()
artist_vertices = artists['id'].unique()
user_artist_edges = user_artists_train[['userID','artistID']]
user_artist_edges = user_artist_edges.sort_values(by = ['userID','artistID'])
user_artist_edges

,userID,artistID
0,2,51
1,2,52
2,2,53
3,2,54
6,2,57
...,...,...
92828,2100,18725
92830,2100,18727
92831,2100,18728
92832,2100,18729


In [63]:
src = list(user_artist_edges['userID'])
dst = list(user_artist_edges['artistID'])
edges = [src,dst]

num_users = int(max(user_vertices)) + 1
num_artists = int(max(artist_vertices)) + 1

interaction_matrix = np.zeros([num_users,num_artists])

for src,dst in zip(edges[0],edges[1]):
    interaction_matrix[src][dst] = 1

In [71]:
interaction_matrix_COO = sparse.coo_matrix(interaction_matrix)